* 队列

先进先出，从右端进，左端出

1. Team Queue [Acwing134](https://www.acwing.com/problem/content/134/)

有 n 个小组要排成一个队列，每个小组中有若干人。
当一个人来到队列时，如果队列中已经有了自己小组的成员，他就直接插队排在自己小组成员的后面，否则就站在队伍的最后面。
请你编写一个程序，模拟这种小组队列。

* 题解
  * 维护 n+1 个队列
  * n个对列用来记录每个小组入队的人
  * 一个队列用于维护这n个队列
    * 如果有新的队列就添加进来
    * 如果出队，就从队头的队列中出队一个元素
    * 如果对头元素为空，就把这个队踢出队列

In [ ]:
from queue import Queue
t = int(input())
p2t = {}
que = [Queue(maxsize=1001) for _ in range(t+1)]           # t+1个队列，最后一个是总队列

index = 1
while t!=0:
    print('Scenario #%d'%index)
    """先把小组人数读进来"""
    for i in range(t):
        temp = list(map(int,input().split()))
        for j in range(1,len(temp)):
            p2t[temp[j]] = i                        # 记录每个小组属于哪个组
    order_line = input()
    """处理指令"""
    while order_line != "STOP":
        if order_line == "DEQUEUE":
            tq = que[-1].queue[0]
            print(tq.get())
            if tq.empty():
                que[-1].get()
        else:
            order,id = order_line.split()
            id = int(id)
            team = p2t[id]
            if que[team].empty():
                que[-1].put(que[team])
            que[team].put(id)
        order_line = input()
    
    t = int(input())
    p2t.clear()
    que = [Queue(maxsize=1001) for _ in range(t+1)]
    index+=1
    print()


2. 蚯蚓 [Acwing133](https://www.acwing.com/problem/content/description/135/)

题目太长了，直接看链接里面的题目描述吧。

* 题解
  * 自己想到了：
    * 关于每秒其他蚯蚓增长的问题，使用一个相对值
    * 使用优先队列存储，每次找到最大的元素  （但是这里会爆时间）
  * 题目解法：
    * 使用3个队列
      1. 存储初始的蚯蚓，并且进行排序
      2. 使用一个left队列，存储每次切开蚯蚓的左边部分
      3. 使用一个right队列，存储每次切开蚯蚓的右边部分
    * **可以证明left队列是单调递减的**（这需要很深入分析数据）
    * **可以证明right队列是单调递减的**（这需要很深入分析数据）


In [ ]:
from queue import Queue
import math
inf = math.inf
n,m,q,u,v,t = list(map(int,input().split()))
p = u/v

qy = list(map(int,input().split()))     # 用来存储蚯蚓
left,right = Queue(),Queue()                      # left队列和right队列

def getmax(qy,left,right):
    cnt1 = qy[0] if qy else -inf
    cnt2 = left.queue[0] if not left.empty() else -inf
    cnt3 = right.queue[0] if not right.empty() else -inf

    if cnt1 > cnt2 and cnt1 > cnt3:
        return qy.pop(0)
    elif cnt2 >= cnt1 and cnt2 >= cnt3:
        return left.get()
    elif cnt3 >= cnt1 and cnt3 >= cnt2:
        return right.get()

qy.sort(reverse=True)                   # 递减排序
deta = 0
for i in range(1,m+1):                  # 从1开始遍历，如果从0开始就会出现 0%t = 0，print的数值与题目不符合
    maxnum = getmax(qy,left,right) + deta
    if i % t == 0:
        print('%d '%maxnum,end='')
    px = math.floor(maxnum*p)
    qx = maxnum - px
    left.put(px-deta - q)
    right.put(qx-deta - q)
    deta = deta + q
print()

for i in range(1,m+n+1):
    maxnum = getmax(qy,left,right)+deta
    if i % t == 0:
        print('%d '%(maxnum),end='')

3. 双端队列

先跳过

4. 单调队列  **很经典的一道题目**

* 思考这种问题
  * 我们需要一个队列/栈来维护一个集合
  * 然后找出这个集合中的极值
  * 发现可以删除无用的数据
  * 发现删除完无用的数据后，集合具有单调性
  * 考虑使用单调队列或者单调栈

4.1 最大子序和 [Acwing135](https://www.acwing.com/problem/content/137/)

给定一个长度为N的整数序列（可能有负数），从中找出一段长度不超过M的连续子序列，使得子序列中所有数的和最大。$N,M \leqslant 3*10^5$

* 题解
  * 连续的和问题可以先转换为前缀和S
  * 问题变成找到两个 i>j 使得 S[i]-S[j] 最大且 i-j <= M
  * 因为两个端点都在变，我们假设右端点从左向右遍历，那么就是在 S[i-m~i-1]中找到一个最小的S[j]，这样的话，以i为右端点的连续子序列的和才最大
  * 暴力解法：对于每个右端点i和它前m长的集合，我们直接进行for循环找到最小的($O(n^2)$)，优化一下可以达到O(nlogn)
  * 仔细分析，发现对于每个i，最小的S[j]前面的样本都是用不到的，那么我们维护每个i的前m个子序列中的最小值就行了，维护的这个序列一定是单调的




In [ ]:
from queue import Queue
from math import inf

n,m = list(map(int,input().split()))
nums = [0]
pre = 0
for num in list(map(int,input().split())):
    nums.append(pre+num)
    pre = nums[-1]

inqueue = []
res = -inf
for i in range(len(nums)):
    # 这一步，判断时候>m个放在前面，是因为对于这个 i，我们只要前m个的
    while inqueue and i-inqueue[0] > m:         # 这一步注意是 i-inqueuqe而不是判断len(inqueue) > m
        inqueue.pop(0)
        
    if inqueue:
        res = max(res,nums[i]-nums[inqueue[0]])
    while inqueue and nums[inqueue[-1]] > nums[i]:
        inqueue.pop()
    inqueue.append(i)
print(res)

In [ ]:
# 这是一个错误代码，与上面的代码进行对比
from queue import Queue
from math import inf

n,m = list(map(int,input().split()))
nums = [0]
pre = 0
for num in list(map(int,input().split())):
    nums.append(pre+num)
    pre = nums[-1]

inqueue = []
res = -inf
for i in range(len(nums)):
    while inqueue and nums[inqueue[-1]] > nums[i]:
        inqueue.pop()
    inqueue.append(i)
    
    while inqueue and i-inqueue[0] > m:         # 这一步注意是 i-inqueuqe而不是判断len(inqueue) > m
        inqueue.pop(0)
    if inqueue:
        res = max(res,nums[i]-nums[inqueue[0]])

print(res)